In [1]:
import os
import django

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "portfolio_management.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()

from common.models import Brokers, Prices, Transactions, FX, Assets
from users.models import CustomUser
from utils import NAV_at_date, Irr, chart_dates, chart_labels, chart_colour, portfolio_at_date, calculate_security_nav, currency_format, calculate_portfolio_value
from datetime import date
from django.db.models import F, Sum

In [2]:
selected_brokers = broker_ids = [2]
effective_current_date = date.today()
currency_target = 'USD'
number_of_digits = 2

CustomUser.objects.get(id=2)

<CustomUser: Yaroslav>

In [3]:
security_id = 7
security = Assets.objects.get(id=security_id)
security.price_at_date(effective_current_date).price

quote = security.prices.filter(date__lte=effective_current_date).order_by('-date').first()
print(currency_target)
if currency_target is not None:
    quote.price = quote.price * FX.get_rate(security.currency, currency_target, effective_current_date)['FX']
quote

USD


<Prices: Long/short is at 24.00000000 on 2020-12-01>

In [11]:
user = CustomUser.objects.get(id=2)

from utils import currency_format_dict_values, format_percentage, summary_over_time

s = summary_over_time(user, effective_current_date, [3], currency_target, number_of_digits)

for index in range(len(s['lines'])):
    if s['lines'][index]['name'] == 'TSR':
        for k, v in s['lines'][index]['data'].items():
            s['lines'][index]['data'][k] = format_percentage(v, digits=1)
    else:
        s['lines'][index] = currency_format_dict_values(s['lines'][index], currency_target, number_of_digits)
    print(s['lines'][index])


1276. Time taken for preparations: 0.03590583801269531
1284. Time taken for getting currencies: 0.029083967208862305
1301. Time taken for getting and preparing transactions: 0.4137859344482422
1323. Time taken for year preparatory calcs for 2010: 0.0
1335. Time taken for calculating invested and cash_out for 2010: 0.0065631866455078125
1352. Time taken for calculating Price change and Capital distribution for 2010: 0.1764528751373291
1360. Time taken for calculating Commission and Tax for 2010: 0.0
1366. Time taken for calculating EoP NAV for 2010: 0.09743571281433105
1371. Time taken for calculating TSR for 2010: 0.13820147514343262
1385. Time taken for calculating FX for 2010: 0.0
1323. Time taken for year preparatory calcs for 2011: 0.0
1335. Time taken for calculating invested and cash_out for 2011: 0.0
1352. Time taken for calculating Price change and Capital distribution for 2011: 0.46419620513916016
1360. Time taken for calculating Commission and Tax for 2011: 0.0
1366. Time tak